In [ ]:
!pip install autokeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 KB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 KB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.8/466.8 KB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 52.8 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_files
import autokeras as ak

In [ ]:
dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz",

    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
    extract=True,
)
# Путь к данным
IMDB_DATADIR=os.path.join(os.path.dirname(dataset),"aclImdb")

classes = ["pos","neg"]
train_data = load_files(
    os.path.join(IMDB_DATADIR, "train"), shuffle=True, categories = classes
)
test_data = load_files(
    os.path.join(IMDB_DATADIR, "test"), shuffle=False, categories = classes
)

x_train = np.array(train_data.data)
y_train = np.array(train_data.target)
x_test = np.array(test_data.data)
y_test = np.array(test_data.target)

print(x_train.shape)
print(y_train.shape)
print(x_train[0][:50])

(25000,)
(25000,)
b'Zero Day leads you to think, even re-think why two'


Второй шаг - запустить TextClassifier

In [ ]:
# Initialize the text classifier
clf = ak.TextClassifier(
    overwrite=True, max_trials=1
) #Только 1 модель для быстрой демонстрации
#Feed the text classifier with training data
clf.fit(x_train, y_train, epochs=2)
#Predict with the best model
predicted_y = clf.predict(x_test)
#Evabulate the best model with testing data
print(clf.evaluate(x_test, y_test))

Trial 1 Complete [00h 06m 04s]
val_loss: 0.2797769606113434

Best val_loss So Far: 0.2797769606113434
Total elapsed time: 00h 06m 04s
Epoch 1/2
782/782 [==============================] - 185s 236ms/step - loss: 0.4339 - accuracy: 0.7786
Epoch 2/2
782/782 [==============================] - 186s 238ms/step - loss: 0.2397 - accuracy: 0.9030


782/782 [==============================] - 51s 65ms/step - loss: 0.2674 - accuracy: 0.8924
[0.26742368936538696, 0.8924000263214111]


In [ ]:
clf.fit(
    x_train,
    y_train,
    validation_split=0.15,
)

In [ ]:
split = 50000
x_val = x_train[split:]
y_val = y_train[split:]
x_train = x_train[:split]
y_train = y_train[:split]
clf.fit(
    x_train,
    y_train,
    epochs=2,
    validation_data=(x_val, y_val),
)

In [ ]:
input_node = ak.TextInput()
output_node = ak.TextBlock(block_type="ngram")(input_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs = input_node, outputs = output_node, overwrite=True, max_trials=1
)
clf.fit(x_train, y_train, epochs=2)

Trial 1 Complete [00h 00m 32s]
val_loss: 0.30051058530807495

Best val_loss So Far: 0.30051058530807495
Total elapsed time: 00h 00m 32s
Epoch 1/2
782/782 [==============================] - 11s 13ms/step - loss: 0.4126 - accuracy: 0.8386
Epoch 2/2
782/782 [==============================] - 7s 8ms/step - loss: 0.2702 - accuracy: 0.8916


In [ ]:
input_node = ak.TextInput()
output_node = ak.TextToIntSequence()(input_node)
output_node = ak.Embedding()(output_node)
#Use separable Conv layers in Keras
output_node = ak.ConvBlock(separable=True)(output_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs = input_node, outputs=output_node, overwrite = True, max_trials=1
)
clf.fit(x_train,y_train,epochs=2)

Trial 1 Complete [00h 01m 18s]
val_loss: 0.6931461691856384

Best val_loss So Far: 0.6931461691856384
Total elapsed time: 00h 01m 18s
Epoch 1/2
782/782 [==============================] - 38s 45ms/step - loss: 0.6932 - accuracy: 0.4959
Epoch 2/2
782/782 [==============================] - 42s 53ms/step - loss: 0.6932 - accuracy: 0.4958


In [ ]:
train_set = tf.data.Dataset.from_tensor_slices(((x_train,),(y_train))).batch(32)
test_set = tf.data.Dataset.from_tensor_slices(((x_test,),(y_test,))).batch(32)

clf = ak.TextClassifier(overwrite=True, max_trials=2)
# Feed the tensorflow Dataset to the classifier
clf.fit(train_set, epochs=2)
#Predict with the best model
predicted_y=clf.predict(test_set)
#Evaluate the best model with the testing data
print(clf.evaluate(test_set))

Trial 2 Complete [00h 01m 30s]
val_loss: 0.468384712934494

Best val_loss So Far: 0.2714446187019348
Total elapsed time: 00h 07m 32s
Epoch 1/2
782/782 [==============================] - 191s 243ms/step - loss: 0.4331 - accuracy: 0.7772
Epoch 2/2
782/782 [==============================] - 182s 233ms/step - loss: 0.2376 - accuracy: 0.9044


782/782 [==============================] - 48s 61ms/step - loss: 0.2846 - accuracy: 0.8845
[0.2845707833766937, 0.8845199942588806]
